In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import xlrd
import os
import csv
from zipfile import ZipFile

In [4]:
datafile = "2013_ERCOT_Hourly_Load_Data"
outfile = "2013_Max_Loads.csv"

In [24]:
def open_zip(datafile):
    with ZipFile('{0}.zip'.format(datafile), 'r') as myzip:
        myzip.extractall()

def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile+'.xls')
    sheet = workbook.sheet_by_index(0)
        
    data = []
    data.append(['Station','Year','Month','Day','Hour','Max Load'])
    stations = sheet.row_values(0, start_colx = 1, end_colx = 9)
    for i in range(len(stations)):
        station_val = sheet.col_values(i+1, start_rowx = 1)
        max_row = station_val.index(max(station_val)) + 1
        Year, Month, Day, Hour, Minute, Second = xlrd.xldate_as_tuple(sheet.cell_value(max_row, 0),0)
        data.append([stations[i],Year, Month, Day, Hour, max(station_val)])
                   
        
    # YOUR CODE HERE
    # Remember that you can use xlrd.xldate_as_tuple(sometime, 0) to convert
    # Excel date to Python tuple of (year, month, day, hour, minute, second)
    return data

def save_file(data, filename):
    # YOUR CODE HERE
    with open(filename, 'w') as sf:
        writer = csv.writer(sf, delimiter = '|')
        for row in data:
            writer.writerow(row)
    
def test():
    open_zip(datafile)
    data = parse_file(datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header
        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

In [25]:
if __name__ == "__main__":
    test()